# Computational Experiments

This notebook showcases the implementation of the three pricing policies (static pricing, iterative pricing, and match-based pricing), applied to the Chicago dataset. We train these policies and subject them to evaluation on both the training and test datasets, all within a specific parameter configuration. For a detailed description and complete results of the experiments please refer to Section 4 of the paper.

## 1. Import packages, classes, functions, and variables

In [1]:
# external packages
import numpy as np
import copy
from gurobipy import *
from collections import defaultdict
import pandas as pd
import pickle
import time

# internal classes, functions, and variables
from lib.network import osm  # construct the network object
from lib.instance_data import InstanceData  # instance data class
from lib.utils import (
    import_demand,  # helper function
    results_saving,  # helper function
    TRAINING_SET_KEY,  # training set keys
    VALIDATION_SET_KEY,  # validation set keys
    TEST_SET_KEY,  # test set keys
    SYN_DATA_KEY,  # synthetic data key
    REAL_DATA_KEY,  # real data key
    MON_PEAK,  # peak time window keys
    SAT_NONPEAK,  # off-peak time window keys
)
from lib.policies import (
    StaticPricingAffineMatchingCombined,  # combined static pricing + affine matching policy
    IterativePricingAffineMatchingCombined,  # combined iterative pricing + affine matching policy
    MatchPricingAffineMatchingCombined,  # combined match-based pricing + affine matching policy
    MatchBasedPricingPolicy,  # pure match-based pricing policy class
    StaticPricingPolicy,  # pure static pricing policy class
    GreedyMatchingPolicy,  # pure greedy matching policy class
    AffineMatchingPolicy,  # pure affine matching policy class
)
from lib.simulation import (
    Simulator,  # the simulator class
    EVALUATION_KEY,  # key of doing evaluation on the synthetic training data
    NO_SAMPLE_KEY,  # key of not doing sampling in the simulation
)

# pricing policy keys
MATCH_KEY = "match"
STATIC_KEY = "static"
ITERATIVE_KEY = "iterative"

## 2. Load the Chicago data

We first load the preprocessed Chicago network data stored in the `network` object, and then calculate the shortest distances between nodes within the network. Following this, we load the preprocessed Chicago demand data (for Monday peak time, 7:30 a.m. to 8:30 a.m.).  In this dataset, we utilize 7 weeks as training data and reserve 1 week for testing purposes. We mainly use the four components of the demand data: `rider_types`, `arrival_rates`, `arrival_types`, `arrival_times`. Please refer to `README.md` for a detailed description of these data.

In [2]:
# import the Chicago network data, and calculate the shortest times between nodes
with open('data/Chicago_network.pickle', 'rb') as f:
    network_data = pickle.load(f)
network = osm(network_data['G'], network_data['edge_keys'])
network.calculate_shortest_times()

# import the real rider demand data in Chicago
time_window = MON_PEAK  # set the time window: MON_PEAK, or SAT_NONPEAK
demand_file = "data/Chicago_demands/{0}_train7_test1.pickle".format(time_window)
rider_types_complete, arrival_rates_complete, arrival_types, arrival_times = import_demand(demand_file)

# Set `n_rider_types`, the number of rider types to be considered in the experiments
# (default: `len(rider_types_complete)`, meaning all rider types are considered).
# The value of `n_rider_types` can be adjusted between 0 and `len(rider_types_complete)` 
# for faster results by focusing on a subset of the rider types.
n_rider_types = len(rider_types_complete)
rider_types = rider_types_complete[:n_rider_types]
arrival_rates = arrival_rates_complete[:n_rider_types]

Failure on properties set() {'osmid'}
Calculating shortest time matrix...
... Done calculating shortest time matrix, 156.27604174613953 seconds


## 3. Parameters setting

Configure parameters for the instance, simulation, and algorithms.

In [4]:
# ----------- instance data -----------
# time_window is already set above
c = 0.9  # cost from 0 to 2
sojourn_time = 300  # average sojourn time in seconds, in {30, 60, 180, 300}

# construct the instance data object
instance_data = InstanceData(
    c,
    sojourn_time,
    time_window,
    network,
    rider_types,
    arrival_rates,
    arrival_types,
    arrival_times,
)


# ------------- simulation parameters ------------
# week numbers for training/validation/test data sets
# (default: [1, 0, 0] for synthetic data; [7, 0, 1] for real data)
data_sets_division = {
    REAL_DATA_KEY: [7, 0, 1],
    SYN_DATA_KEY: [1, 0, 0],
}
# get the test week number
test_week = data_sets_division[REAL_DATA_KEY][TEST_SET_KEY - 1]

# simulation time for constraint sampling
# (default: 1e+5 seconds for c=0.7, 0.9 in synthetic training set;
# 2e+5 seconds for c=1.1 in synthetic training set;
# 3600 seconds for test set which is real data so it is shorter)
time_limits = {TEST_SET_KEY: 3600}
if c < 1:
    time_limits[TRAINING_SET_KEY] = 100000
else:
    time_limits[TRAINING_SET_KEY] = 500000
# parameter to control the observing rate for constraint sampling,
# which is a portion rate of the summation of total arrival rate and abandoning rate (default: 0.1)
observer_rate_portion = 0.1
# a sufficiently long time for evaluating the synthetic training set (default: 2e+6 seconds)
evaluation_time = 2000000
# simulation time for greedy matching (to control the sample sizeg) (default: 2e+4 seconds)
time_limit_greedy = 20000
# number of repetitions for test set (default: 1 for synthetic training set; 10 for test set)
repeat_times = {
    TRAINING_SET_KEY: 1,
    TEST_SET_KEY: 10,
}

# store the simulation-specific paramters in a dictionary
sim_params = {
    "data_sets_division": data_sets_division,
    "observer_rate_portion": observer_rate_portion,
    "time_limits": time_limits,
    "evaluation_time": evaluation_time,
    "time_limit_greedy": time_limit_greedy,
    "repeat_times": repeat_times,
}


# ------------- algorithm parameters -------------
# rounds of bootstrapping for constraint sampling (default: 5)
bootstrapping_rounds = 5
# the default static conversion (uniform for all rider types) for an arbitrary static pricing
static_conversion_value = 0.5
# termination gap for cutting plane method
EPSILON_CUTTING_PLANE = 0.5
# termination gap for subgradient method
EPSILON_SUBGRADIENT = 1e-5

# store the algorithm-specific paramters in a dictionary
alg_params = {
    "bootstrapping_rounds": bootstrapping_rounds,
    "static_conversion_value": static_conversion_value,
    "EPSILON_CUTTING_PLANE": EPSILON_CUTTING_PLANE,
    "EPSILON_SUBGRADIENT": EPSILON_SUBGRADIENT,
}

## 4. Policy training

We train three policies, wherein pricing and matching policies are trained concurrently. During this process, we generate a synthetic training set under an extended time period leveraging the arrival rates estimated from actual training data. This approach ensures that sufficiently many constraints are sampled to derive high-quality solutions.
- *Static pricing*: We solve a fluid mathematical program to determine static prices (stored in `static_conversions[STATIC_KEY]`). Simultaneously, we optimize the matching policy (stored in `affine_weights[STATIC_KEY]`) by solving the ALP (Approximate Linear Program).  
- *Iterative pricing*: Starting at an initial price, we optimize the affine matching policy by solving the ALP. Then we update the costs by simulation, followed by adjustments to the prices and affine weights. Repeat this cycle until convergence, resulting in the converged static prices (saved in `static_conversions[ITERATIVE_KEY]`) and matching policy (saved in `affine_weights[ITERATIVE_KEY]`).
- *Match-based pricing*: We employ a method outlined in Section 3.2 of the paper to solve the ALP and obtain optimal affine weights, which are saved in `affine_weights[MATCH_KEY]`.

In [5]:
# initialize the simulator instance: use_real_data = False for synthetic data
simulator_training = Simulator(instance_data, sim_params, use_real_data=False)

# generate the synthetic training set
simulator_training.events_generator(
    time_limit=simulator_training.time_limits[TRAINING_SET_KEY]
)

# policy results
# record the affine weights for each policy
affine_weights = dict()
# record the optimal static conversions for static/iterative policies
static_conversions = dict()
# record the sample sizes for each policy
sample_sizes = dict()
# record the training time for each policy
training_time = dict()

# --------------- Policy training ---------------
policy = STATIC_KEY
SPAMC = StaticPricingAffineMatchingCombined(instance_data, alg_params)
(
    static_conversions[policy],
    affine_weights[policy],
    static_profit_UB,
    sample_sizes[policy],
    training_time[policy],
) = SPAMC.training_static(simulator_training)
print(
    "Static pricing policy is trained. Training time: {0} seconds.".format(
        training_time[policy]
    )
)

policy = ITERATIVE_KEY
IPAMC = IterativePricingAffineMatchingCombined(instance_data, alg_params)
(
    static_conversions[policy],
    affine_weights[policy],
    sample_sizes[policy],
    training_time[policy],
) = IPAMC.training_iterative(simulator_training)
print(
    "Iterative pricing policy is trained. Training time: {0} seconds.".format(
        training_time[policy]
    )
)

policy = MATCH_KEY
MPAMC = MatchPricingAffineMatchingCombined(instance_data, alg_params)
(
    affine_weights[policy],
    sample_sizes[policy],
    training_time[policy],
) = MPAMC.training_match(
    simulator_training,
    static_profit_UB,
    static_conversions[STATIC_KEY],
    affine_weights[STATIC_KEY],
)
print(
    "Match-based pricing policy is trained. Training time: {0} seconds.".format(
        training_time[policy]
    )
)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-24


Static pricing policy is trained. Training time: 9027.504397392273 seconds.
Iterative pricing policy is trained. Training time: 27798.425683021545 seconds.
Match-based pricing policy is trained. Training time: 39095.36449408531 seconds.


## 5. Performance evaluation

We evaluate the performance of the trained policies on both the training set and the test set. Here we store the trained policy classes in `pricing_policies` and `matching_policies` dictionaries. (Additionally, it's worth noting that you can have the flexibility to implement custom pricing and matching policies in the framework (by creating classes that inherit from classes `PricingPolicy` or `MatchingPolicy`). Detailed information on the format and structure of these classes can be found in the  `README.md` and `/lib/policies.py`.)

In [6]:
# store three policies in pricing and matching policies dictionaries
pricing_policies = dict()
matching_policies = dict()

for policy in [STATIC_KEY, ITERATIVE_KEY, MATCH_KEY]:
    if policy in [STATIC_KEY, ITERATIVE_KEY]:
        pricing_policies[policy] = StaticPricingPolicy(
            instance_data, static_conversions[policy]
        )
    else:
        pricing_policies[policy] = MatchBasedPricingPolicy(
            instance_data, affine_weights[policy]
        )
    matching_policies[policy] = AffineMatchingPolicy(
        instance_data, affine_weights[policy]
    )

In our evaluation process, we consider a set of essential performance metrics: profit (\\$/min), average quoted price (\\$/mile $\cdot$ rider), average payment (\\$/mile $\cdot$ request), throughput (requests/min), match rate, and cost efficiency. Please refer to Section 4 of the paper for the detailed explanationd of these metrics. These metrics are saved in the dictionary `metrics`. To access the results, you should specify the policy key and data set type key. Additionally, we retain the comprehensive details of each rider arrival in pandas dataframes, which are stored in the `eyeball_metrics` dictionary. Here we initialize these two dictionaries.

In [7]:
# metrics initialization
# metrics to be considered
metrics_list = [
    "profit",
    "ave_quoted_price",
    "ave_payment",
    "throughput",
    "match_rate",
    "cost_efficiency",
]
# metrics for each policy under all repetitions (test set only)
metrics_repeats = defaultdict(defaultdict)
# average metrics for each policy
metrics = defaultdict(defaultdict)
# eyeball-level metrics in pandas dataframes, each row being a rider
eyeball_metrics = defaultdict(defaultdict)
for policy in [ITERATIVE_KEY, STATIC_KEY, MATCH_KEY]:
    for set_type in [TRAINING_SET_KEY, VALIDATION_SET_KEY, TEST_SET_KEY]:
        metrics_repeats[policy][set_type] = {i: list() for i in metrics_list}
        metrics[policy][set_type] = {
            "profit": 0,
            "ave_quoted_price": 1,
            "ave_payment": np.nan,
            "throughput": np.nan,
            "match_rate": np.nan,
            "cost_efficiency": np.nan,
        }
        eyeball_metrics[policy][set_type] = list()

### 5.1. Training performance

TWe subject the trained policies to evaluation using the synthetic training dataset, extending the simulation time to `evaluation_time = 2,000,000` seconds.

In [8]:
# set the data set type as training set
set_type = TRAINING_SET_KEY

# record the evaluation time
start_time = time.time()

# initialize the simulator instance: use_real_data = False for synthetic data
simulator_training_evaluate = Simulator(instance_data, sim_params, use_real_data=False)

# generate the synthetic training set for evaluation which is sufficiently long
simulator_training_evaluate.events_generator(
    time_limit=simulator_training_evaluate.evaluation_time,
    seed=0
)

# training performance evaluation for each policy
for policy in [STATIC_KEY, ITERATIVE_KEY, MATCH_KEY]:
    # static and iterative pricing policies are evaluated only when the optimal static
    # profit is positive (conversions are non-zero)
    if (policy in [STATIC_KEY, ITERATIVE_KEY] and static_profit_UB > 0) or (
        policy == MATCH_KEY
    ):
        simulator_training_evaluate.simulation(
            pricing_policies[policy],
            matching_policies[policy],
            data_set_key=set_type,
            sample_key=NO_SAMPLE_KEY,
            seed=0,
        )

        # record system level metrics
        for key in metrics_list:
            if key == "profit":
                metrics[policy][set_type][key] = (
                    simulator_training_evaluate.metrics_list[-1][key] * 60 / 1609
                )  # transfer to per min and mile  (originally in second and meter)
            elif key == "throughput":
                metrics[policy][set_type][key] = (
                    simulator_training_evaluate.metrics_list[-1][key] * 60
                )  # transfer to per min (originally in second)
            else:
                metrics[policy][set_type][
                    key
                ] = simulator_training_evaluate.metrics_list[-1][key]
        # record rider level metrics in dataframes
        eyeball_metrics[policy][set_type].append(
            simulator_training_evaluate.eyeball_metrics_df_list[-1]
        )

end_time = time.time()
print(
    "Training performance evaluation is done. Time: {0} seconds.".format(
        end_time - start_time
    )
)

Training performance evaluation is done. Time: 951.8589758872986 seconds.


### 5.2. Test performance

We conduct evaluations on the actual test dataset, averaging the results over 10 simulation runs.

In [9]:
# set the data set type as test set
set_type = TEST_SET_KEY

# record the evaluation time
start_time = time.time()

# initialize the SharedPricing instance: use_real_data = True for real data
simulator_test = Simulator(instance_data, sim_params, use_real_data=True)

# evaluate the performance of each policy on the test set
for policy in [STATIC_KEY, ITERATIVE_KEY, MATCH_KEY]:
    # static and iterative pricing policies are only evaluated when the optimal static
    # profit is positive (conversions are non-zero)
    if (policy in [STATIC_KEY, ITERATIVE_KEY] and static_profit_UB > 0) or (
        policy == MATCH_KEY
    ):
        # repeat (10 times by default) for each policy
        for it in range(repeat_times[set_type]):
            # randomly generate the departure and conversions on the test set for each repetition
            simulator_test.events_generator(
                time_limit=simulator_test.time_limits[set_type], seed=it
            )

            # run the simulation based on the given policy
            simulator_test.simulation(
                pricing_policies[policy],
                matching_policies[policy],
                data_set_key=set_type,
                sample_key=NO_SAMPLE_KEY,
                seed=it,
            )

            # record system level metrics
            for key in metrics_list:
                if key == "profit":
                    metrics_repeats[policy][set_type][key].append(
                        simulator_test.metrics_list[-1][key] * 60 / 1609 / test_week
                    )  # transfer to per min and mile  (originally in second and meter)
                elif key == "throughput":
                    metrics_repeats[policy][set_type][key].append(
                        simulator_test.metrics_list[-1][key] * 60 / test_week
                    )  # transfer to per min (originally in second)
                else:
                    metrics_repeats[policy][set_type][key].append(
                        simulator_test.metrics_list[-1][key]
                    )
            # record rider level metrics
            eyeball_metrics[policy][set_type].append(
                simulator_test.eyeball_metrics_df_list[-1]
            )

        # calculate average metrics over the repetitions
        for key in metrics_list:
            metrics[policy][set_type][key] = np.mean(
                metrics_repeats[policy][set_type][key]
            )

end_time = time.time()
print(
    "Test performance evaluation is done. Time: {0} seconds.".format(
        end_time - start_time
    )
)

Test performance evaluation is done. Time: 18.141817808151245 seconds.


## 6. Save and print the results

We store all the policies-related arguments and performance metric results in the `results` dictionary. Additionally, for a clear comparative evaluation of these policies, we present the training and test performance metrics in the pandas dataframe `metrics_df` under the parameters of $c=0.9, \theta=1/5$, which have consistent results with the corresponding columns in Tables 2 and EC.2 of the paper.

In [10]:
# save all policies and results data in a dictionary
results = results_saving(
    metrics,  # the final results of metrics under three policies on training/test sets
    eyeball_metrics,  # the detail information for all rider arrivals
    training_time,  # the running time for training
    affine_weights,  # the optimal affine weights for all pricing policies
    static_conversions,  # the optimal conversions for static/iterative pricing policy
    sample_sizes,  # the sample sizes for all pricing policies
    static_profit_UB,  # the optimal profit for static/iterative pricing policy
)

# organize the performance metrics results in dataframes
metrics_df = dict()
for set_type in [TRAINING_SET_KEY, TEST_SET_KEY]:
    if set_type == TRAINING_SET_KEY:
        metrics_df[set_type] = pd.DataFrame.from_dict(
            [
                metrics[ITERATIVE_KEY][TRAINING_SET_KEY],
                metrics[STATIC_KEY][TRAINING_SET_KEY],
                {"profit": static_profit_UB},
                metrics[MATCH_KEY][TRAINING_SET_KEY],
            ]
        ).T.rename(
            columns={0: ITERATIVE_KEY, 1: STATIC_KEY, 2: "static_UB", 3: MATCH_KEY}
        )
    else:
        metrics_df[set_type] = pd.DataFrame.from_dict(
            [
                metrics[ITERATIVE_KEY][TEST_SET_KEY],
                metrics[STATIC_KEY][TEST_SET_KEY],
                metrics[MATCH_KEY][TEST_SET_KEY],
            ]
        ).T.rename(columns={0: ITERATIVE_KEY, 1: STATIC_KEY, 2: MATCH_KEY})
    metrics_df[set_type]["match vs iterative"] = (
        metrics_df[set_type][MATCH_KEY] - metrics_df[set_type][ITERATIVE_KEY]
    ) / metrics_df[set_type][ITERATIVE_KEY]
    metrics_df[set_type]["match vs static"] = (
        metrics_df[set_type][MATCH_KEY] - metrics_df[set_type][STATIC_KEY]
    ) / metrics_df[set_type][STATIC_KEY]
    if set_type == TRAINING_SET_KEY:
        metrics_df[set_type]["match vs static_UB"] = (
            metrics_df[set_type][MATCH_KEY] - metrics_df[set_type]["static_UB"]
        ) / metrics_df[set_type]["static_UB"]

In [11]:
print(
    "Training results when time window = {1}, c = {0}, and sojourn time = {2} seconds:".format(
        c, time_window, sojourn_time
    )
)
metrics_df[TRAINING_SET_KEY]

Training results when time window = MON-PEAK, c = 0.9, and sojourn time = 300 seconds:


,iterative,static,static_UB,match,match vs iterative,match vs static,match vs static_UB
profit,2.320160,2.559291,2.587818,2.984788,0.286458,0.166256,0.153399
ave_quoted_price,0.864328,0.826483,NaN,0.822395,-0.048515,-0.004946,NaN
ave_payment,0.842847,0.806512,NaN,0.786973,-0.066292,-0.024226,NaN
throughput,3.845100,4.913160,NaN,5.024970,0.306850,0.022757,NaN
match_rate,0.589748,0.614598,NaN,0.664123,0.126114,0.080581,NaN
cost_efficiency,0.215759,0.230143,NaN,0.270939,0.255748,0.177264,NaN


In [12]:
print(
    "Test results when time window = {1}, c = {0}, and sojourn time = {2} seconds:".format(
        c, time_window, sojourn_time
    )
)
metrics_df[TEST_SET_KEY]

Test results when time window = MON-PEAK, c = 0.9, and sojourn time = 300 seconds:


,iterative,static,match,match vs iterative,match vs static
profit,2.199755,2.558417,2.996596,0.362241,0.171270
ave_quoted_price,0.866908,0.829754,0.825062,-0.048271,-0.005655
ave_payment,0.845201,0.810280,0.790574,-0.064631,-0.024320
throughput,3.755000,4.846667,4.953333,0.319130,0.022008
match_rate,0.573888,0.613074,0.661052,0.151882,0.078258
cost_efficiency,0.205017,0.224696,0.264982,0.292491,0.179293
